<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej3FuncionesActivacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Cargar el conjunto de datos Iris
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
# Codificar las etiquetas en formato one-hot para representar las clases de manera binaria clase 1 -> [1 0 0], clase 2-> [0 1 0] ....
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Definir funciones de activación
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def leaky_relu(x, alpha=0.01):
    return np.where(x >= 0, x, alpha * x)

In [ ]:
# Función para entrenar y evaluar una red neuronal con una función de activación dada
def train_and_evaluate(activation_function):
    np.random.seed(42)  # Para reproducibilidad

    input_size = X_train.shape[1]
    hidden_size = 5
    output_size = y_train.shape[1]
    learning_rate = 0.001
    epochs = 1000

    # Inicialización de pesos y sesgos
    weights_input_hidden = np.random.randn(input_size, hidden_size)
    bias_hidden = np.zeros((1, hidden_size))
    weights_hidden_output = np.random.randn(hidden_size, output_size)
    bias_output = np.zeros((1, output_size))

    for epoch in range(epochs):
        # Feedforward
        hidden_output = activation_function(np.dot(X_train, weights_input_hidden) + bias_hidden)
        predicted_output = softmax(np.dot(hidden_output, weights_hidden_output) + bias_output)

        # Backpropagation
        error = y_train - predicted_output
        delta_output = error * (predicted_output * (1 - predicted_output))
        delta_hidden = delta_output.dot(weights_hidden_output.T) * (hidden_output * (1 - hidden_output))

        # Actualizar pesos y sesgos
        weights_hidden_output += hidden_output.T.dot(delta_output) * learning_rate
        bias_output += np.sum(delta_output, axis=0, keepdims=True) * learning_rate
        weights_input_hidden += X_train.T.dot(delta_hidden) * learning_rate
        bias_hidden += np.sum(delta_hidden, axis=0, keepdims=True) * learning_rate

    # Evaluación en datos de prueba
    hidden_output_test = activation_function(np.dot(X_test, weights_input_hidden) + bias_hidden)
    predicted_output_test = softmax(np.dot(hidden_output_test, weights_hidden_output) + bias_output)

    accuracy = np.mean(np.argmax(predicted_output_test, axis=1) == np.argmax(y_test, axis=1))
    return accuracy

Recordemos, El término "accuracy" se refiere a la precisión de un modelo de clasificación, es decir, la proporción de ejemplos clasificados correctamente en relación con el total de ejemplos. En otras palabras, la precisión mide qué tan cerca está el modelo de clasificación de predecir correctamente las etiquetas de las muestras.

Por ejemplo, si tienes 100 muestras en total y tu modelo clasifica correctamente 85 de ellas, la precisión sería 85100=0.8510085​=0.85, lo que equivale al 85%.

In [ ]:
# Función de activación Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [ ]:
# Probar y comparar las funciones de activación
activation_functions = [sigmoid, tanh, relu, leaky_relu]
for activation_function in activation_functions:
    accuracy = train_and_evaluate(activation_function)
    activation_name = activation_function.__name__
    print(f'Activación: {activation_name}, Precisión: {accuracy:.4f}')